# Load and test network on any input

In [1]:
import torch
import numpy as np

import poker_env.datatypes as pdt
from poker_env.env import Poker
from poker_env.config import Config
from models.networks import OmahaActor,OmahaQCritic,OmahaObsQCritic,CombinedNet
from models.model_layers import ProcessHandBoard
from models.model_utils import strip_padding,unspool,hardcode_handstrength

In [2]:
config = Config()
game_object = pdt.Globals.GameTypeDict[pdt.GameTypes.OMAHAHI]

env_params = {
    'game':pdt.GameTypes.OMAHAHI,
    'betsizes': game_object.rule_params['betsizes'],
    'bet_type': game_object.rule_params['bettype'],
    'n_players': 2,
    'pot':1,
    'stacksize': game_object.state_params['stacksize'],
    'cards_per_player': game_object.state_params['cards_per_player'],
    'starting_street': game_object.starting_street,
    'global_mapping':config.global_mapping,
    'state_mapping':config.state_mapping,
    'obs_mapping':config.obs_mapping,
    'shuffle':True
}

env = Poker(env_params)

nS = env.state_space
nA = env.action_space
nB = env.betsize_space
seed = 1235
device = torch.device(cuda_dict[args.gpu] if torch.cuda.is_available() else "cpu")
gpu1 = 'cuda:0'
gpu2 = 'cuda:1'
network_params                                = config.network_params
network_params['device']                      = device

# Instantiate net and copy weights

In [3]:
def copy_weights(net,path):
    if torch.cuda.is_available():
        layer_weights = torch.load(path)
    else:
        layer_weights = torch.load(path,map_location=torch.device('cpu'))
    for name, param in net.named_parameters():
        if name in layer_weights:
            print('update_weights',name)
            param.data.copy_(layer_weights[name].data)
            param.requires_grad = False

In [4]:
net = ProcessHandBoard(network_params,hand_length=4)

In [5]:
copy_weights(net,config.network_params['actor_hand_recognizer_path'])

update_weights suit_conv.0.weight
update_weights suit_conv.0.bias
update_weights suit_conv.1.weight
update_weights suit_conv.1.bias
update_weights rank_conv.0.weight
update_weights rank_conv.0.bias
update_weights rank_conv.1.weight
update_weights rank_conv.1.bias
update_weights hidden_layers.0.weight
update_weights hidden_layers.0.bias
update_weights hidden_layers.1.weight
update_weights hidden_layers.1.bias
update_weights categorical_output.weight
update_weights categorical_output.bias


In [6]:
for name, param in net.named_parameters():
    print(name,param.requires_grad)

suit_conv.0.weight False
suit_conv.0.bias False
suit_conv.1.weight False
suit_conv.1.bias False
rank_conv.0.weight False
rank_conv.0.bias False
rank_conv.1.weight False
rank_conv.1.bias False
hand_out.weight True
hand_out.bias True
hidden_layers.0.weight False
hidden_layers.0.bias False
hidden_layers.1.weight False
hidden_layers.1.bias False
categorical_output.weight False
categorical_output.bias False


In [7]:
net_input = torch.tensor([[[13.,  4., 11.,  3.,  5.,  1., 14.,  2., 11.,  2., 14.,  1.,  2.,  3.,
           9.,  2., 13.,  3.],
         [13.,  4., 11.,  3.,  5.,  1., 14.,  2., 11.,  2., 14.,  1.,  2.,  3.,
           9.,  2., 13.,  3.]]])

In [8]:
net_input.size()

torch.Size([1, 2, 18])

In [10]:
def compare_weights(net):
    path = '/Users/morgan/Code/PokerAI/poker/checkpoints/frozen_layers/hand_board_weights'
    if torch.cuda.is_available():
        layer_weights = torch.load(path)
    else:
        layer_weights = torch.load(path,map_location=torch.device('cpu'))
    print(net)
    for name, param in net.named_parameters():
        if name in layer_weights:
            print(f'Layer {name},Equal {np.array_equal(param.data.numpy(),layer_weights[name].data.numpy())}')


# compare vs baseline

In [11]:
compare_weights(net)

ProcessHandBoard(
  (suit_conv): Sequential(
    (0): Conv1d(5, 128, kernel_size=(1,), stride=(1,))
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (rank_conv): Sequential(
    (0): Conv1d(5, 128, kernel_size=(5,), stride=(1,))
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (hand_out): Linear(in_features=7463, out_features=128, bias=True)
  (hidden_layers): ModuleList(
    (0): Linear(in_features=16, out_features=32, bias=True)
    (1): Linear(in_features=32, out_features=32, bias=True)
  )
  (bn_layers): ModuleList()
  (categorical_output): Linear(in_features=4096, out_features=7463, bias=True)
)
Layer suit_conv.0.weight,Equal True
Layer suit_conv.0.bias,Equal True
Layer suit_conv.1.weight,Equal True
Layer suit_conv.1.bias,Equal True
Layer rank_conv.0.weight,Equal True
Layer rank_conv.0.bias,Equal True
Layer rank_conv.1.weight,Equal T

In [12]:
out = net(net_input)

tensor([[ 0.1641],
        [ 0.4222],
        [-0.3495],
        [-0.5700],
        [ 0.3859]])
x tensor([[[13.,  4., 11.,  3.,  5.,  1., 14.,  2., 11.,  2., 14.,  1.,  2.,  3.,
           9.,  2., 13.,  3.],
         [13.,  4., 11.,  3.,  5.,  1., 14.,  2., 11.,  2., 14.,  1.,  2.,  3.,
           9.,  2., 13.,  3.]]])
tensor([[ 0.1641],
        [ 0.4222],
        [-0.3495],
        [-0.5700],
        [ 0.3859]])
result torch.Size([1, 2, 60])
best hand guess tensor([[2469, 2469]])
baseline tensor([[[2469.],
         [2469.]]])


NameError: name 'asdf' is not defined

In [ ]:
print(hardcode_handstrength(net_input))